<a href="https://colab.research.google.com/github/villasen/colab_notebooks/blob/master/10_KWS_speech_Quantization_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/villasen/Speech-tiny.git
!rm -r sample_data
!rm Speech-tiny/.gitignore
!rm -r Speech-tiny/.git

Cloning into 'Speech-tiny'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 54192 (delta 1), reused 6 (delta 1), pack-reused 54185
Receiving objects: 100% (54192/54192), 1.40 GiB | 28.64 MiB/s, done.
Resolving deltas: 100% (137/137), done.
Checking out files: 100% (55253/55253), done.


In [0]:
MFCC=10
# If running models build locally
LOCAL_MODELS = False

In [3]:
import keras
from keras import layers
from keras import models
from keras import backend as K
from keras import optimizers
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np
import librosa
import librosa.display
import tensorflow
import scipy
import os, shutil
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from six.moves import urllib
import sys
import tarfile
import os.path
from os import path
from tensorflow.python.ops import io_ops
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
import matplotlib.pyplot as plt
import librosa.display
import pandas as pd
import seaborn as sns; sns.set()

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from numpy  import array
from tensorflow.contrib.quantize.python import fold_batch_norms
from tensorflow.contrib.quantize.python import quantize
from tensorflow.python.framework import ops
print(tf.__version__)



########### FUNCTIONS

def urban_wav2mfcc(file_path, max_pad_len):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=MFCC, n_fft=640, hop_length=320)
      
    pad_width = max_pad_len - mfcc.shape[1]
    #print(max_pad_len)
    #print(mfcc.shape[1])
    #print(pad_width)
    if pad_width < 0: 
      print(mfcc.shape[1])
      print(pad_width)
      print("error in "+ file_path)
    
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc 


  
  
def load_wav_file(wav_file):
  with tf.Session(graph=tf.Graph()) as sess:
      wav_filename_placeholder = tf.placeholder(tf.string, [])
      wav_loader = io_ops.read_file(wav_filename_placeholder)
      wav_decoder = contrib_audio.decode_wav(wav_loader, desired_channels=1)
      return sess.run(
          wav_decoder,
          feed_dict={wav_filename_placeholder: wav_file}).audio.flatten()


def get_mfccs(file):
  try:
      wave, sr = librosa.load(wavfile , mono=True, sr=None)
      mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=MFCC, n_fft=640, hop_length=320, norm='ortho')
      mfcc_scaled = np.mean(mfcc.T, axis=0)
      
  except Exception as error:
    print("Error found handling file: ", file)
    return None
  
  return mfcc_scaled





class AudioAugmentation:
    def read_audio_file(self, file_path):
        input_length = 16000
        data = librosa.core.load(file_path)[0]
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data

    def write_audio_file(self, file, data, sample_rate=16000):
        librosa.output.write_wav(file, data, sample_rate)

    def add_noise(self, data):
            noise = np.random.randn(len(data))
            data_noise = data + 0.005 * noise
            return data_noise
    def shift(self, data):
            return np.roll(data, 1600)
    def stretch(self, data, rate=1):
            input_length = 16000
            data = librosa.effects.time_stretch(data, rate)
            if len(data) > input_length:
                data = data[:input_length]
            else:
                data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
            return data
    def plot_time_series(self, data):
            fig = plt.figure(figsize=(10, 10))
            plt.title('Raw wave ')
            plt.ylabel('Amplitude')
            plt.plot(np.linspace(0, 1, len(data)), data)
            plt.show()

Using TensorFlow backend.


1.15.0


In [6]:
import tensorflow as tf
tf.enable_eager_execution()
import os.path
from os import path



DATA_PATH= '/content/Speech-tiny/dataset/'



labels = os.listdir('Speech-tiny/dataset/')
print(labels)

test_single_file = '/content/test_single_file/'
if path.exists('/content/test_single_file/') :
    print ("folder test_single exits, removing")
    !rm -rf /content/test_single_file
os.mkdir(test_single_file)

target_npy_files = "/content/target_npy_files/"
if path.exists("/content/target_npy_files/") :
    print ("folder target_npy_files exits, removing")
    !rm -rf /content/target_npy_files
os.mkdir(target_npy_files)


target_txt_files = "/content/target_txt_files/"
if path.exists("/content/target_txt_files/") :
    print ("folder target_txt_files folder exits, removing")
    !rm -rf /content/target_txt_files
os.mkdir(target_txt_files)




# follow
if path.exists("/content/test_single_file/follow") :
     print ("folder test_single_file/follow exits, removing")
     !rm -rf /content/test_single_file/follow
os.mkdir('/content/test_single_file/follow')



# forward
if path.exists("/content/test_single_file/forward") :
     print ("folder test_single_file/forward exits, removing")
     !rm -rf /content/test_single_file/forward
os.mkdir('/content/test_single_file/forward')

# go
if path.exists("/content/test_single_file/go") :
    print ("folder test_single_file/go exits, removing")
    !rm -rf /content/test_single_file/go
os.mkdir('/content/test_single_file/go')

# yes
if path.exists("/content/test_single_file/yes") :
    print ("folder test_single_file/yes exits, removing")
    !rm -rf /content/test_single_file/yes
os.mkdir('/content/test_single_file/yes')

# no
if path.exists("/content/test_single_file/no") :
    print ("folder test_single_file/no exits, removing")
    !rm -rf /content/test_single_file/no
os.mkdir('/content/test_single_file/no')


# stop
if path.exists("/content/test_single_file/stop") :
    print ("folder test_single_file/stop exits, removing")
    !rm -rf /content/test_single_file/stop
os.mkdir('/content/test_single_file/stop')


# right
if path.exists("/content/test_single_file/right") :
    print ("folder test_single_file/right exits, removing")
    !rm -rf /content/test_single_file/right
os.mkdir('/content/test_single_file/right')


# left
if path.exists("/content/test_single_file/left") :
    print ("folder test_single_file/left exits, removing")
    !rm -rf /content/test_single_file/left
os.mkdir('/content/test_single_file/left')


# up
if path.exists("/content/test_single_file/up") :
    print ("folder test_single_file/up exits, removing")
    !rm -rf /content/test_single_file/up
os.mkdir('/content/test_single_file/up')


# down
if path.exists("/content/test_single_file/down") :
    print ("folder test_single_file/down exits, removing")
    !rm -rf /content/test_single_file/down
os.mkdir('/content/test_single_file/down')


if path.exists("/content/models/") :
    print ("folder /content/models/ exits, removing")
    !rm -rf /content/models/
os.mkdir('/content/models/')

if path.exists("/content/models/speech") :
    print ("folder /content/models/speech exits, removing")
    !rm -rf /content/models/speech/
os.mkdir('/content/models/speech')

['bed', 'off', 'yes', 'forward', 'dog', 'left', 'right', 'on', 'follow', 'up', 'sheila', 'learn', 'backward', 'house', 'cat', 'no', 'stop', 'go', 'down']
folder test_single exits, removing
folder target_npy_files exits, removing
folder target_txt_files folder exits, removing
folder /content/models/ exits, removing


In [0]:
#Create instance for augmentation
aug = AudioAugmentation()

# read file and show sound graph
data = aug.read_audio_file("/content/Speech-tiny/dataset/forward/012187a4_nohash_0.wav")
aug.plot_time_series(data)

# Add noise
data_noise = aug.add_noise(data)
aug.plot_time_series(data_noise)

# shift sound
data_shift = aug.shift(data)
aug.plot_time_series(data_shift)

# strech sound 
data_stretch = aug.stretch(data)
aug.plot_time_series(data_stretch)



In [0]:
# Data Augmentation 
#Create instance for augmentation
aug = AudioAugmentation()

path = DATA_PATH
labels = os.listdir(path) 
print(labels)
i=0
for label in labels:
  mfcc_vectors=[]
   
  wavefiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
  if label=='follow' or label=='forward' or label=='go' or label=='yes' or label=='no' or label=='stop' or label=='right' or label=='left' or label=='up' or label=='down': 
    #print(wavefiles)
    for wavfile in wavefiles:
        wave, sr = librosa.load(wavfile , mono=True, sr=None)
        data_noise = aug.add_noise(wave)
        #data_files = np.pad(data_noise, pad_width=((0, 0), (0, pad_width)), mode='constant')
        #data_vectors.append(data_files)
        num =str(i)
        #np.save( wavfile + '_' + num, data_noise)
        aug.write_audio_file( '/content/Speech-tiny/dataset/' + label + '/' + 'augmented_' + num + '.wav'  , data_noise)
        i=i+1

    print(label)

In [0]:
path = DATA_PATH
labels = os.listdir(path) 
print(labels)

max_feature = 0
for label in labels:
  mfcc_vectors=[]
   
  wavefiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
    
  if label=='follow' or label=='forward' or label=='go' or label=='yes' or label=='no' \
  or label=='stop' or label=='right' or label=='left' or label=='up' or label=='down':
#  if label=='clapping': 
    
    for wavfile in wavefiles:
        max_pad_len = 51



        wave, sr = librosa.load(wavfile , mono=True, sr=None)
        mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=MFCC, n_fft=640, hop_length=320, norm='ortho')
        #mfcc = np.mean(librosa.feature.mfcc(wave, sr=16000, n_mfcc=MFCC, n_fft=640, hop_length=320, norm='ortho'))



        #print(mfcc)

        pad_width = max_pad_len - mfcc.shape[1]

        #print(max_pad_len)
        #print(mfcc.shape[1])
        #print(pad_width)

        if pad_width < 0: 
          print(mfcc.shape[1])
          print(pad_width)
          print("error in "+ file_path)
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        mfcc_vectors.append(mfcc)


    np.save('/content/target_npy_files/' + label + '.npy', mfcc_vectors)
    np.savetxt('/content/target_txt_files/' + label + '.txt', mfcc_vectors[2], delimiter=', ')
    print(label)

In [7]:
!cp /content/Speech-tiny/npy_files_Augmented/*.npy /  /content/target_npy_files
!cp /content/drive/My\ Drive/ML_models/Tensorflow_lite/Speech/10_word_ESC50_split/speech-model.tflite   /content/models/speech/
!cp /content/drive/My\ Drive/ML_models/Tensorflow_lite/Speech/10_word_ESC50_split/speech_model_quant.tflite   /content/models/speech/
!cp /content/drive/My\ Drive/ML_models/Tensorflow_lite/Speech/10_word_ESC50_split/speech-model_quant_io.tflite   /content/models/speech/

cp: -r not specified; omitting directory '/'
cp: cannot stat '/content/drive/My Drive/ML_models/Tensorflow_lite/Speech/10_word_ESC50_split/speech-model.tflite': No such file or directory
cp: cannot stat '/content/drive/My Drive/ML_models/Tensorflow_lite/Speech/10_word_ESC50_split/speech_model_quant.tflite': No such file or directory
cp: cannot stat '/content/drive/My Drive/ML_models/Tensorflow_lite/Speech/10_word_ESC50_split/speech-model_quant_io.tflite': No such file or directory


In [0]:
split_ratio = 0.9
random_state = 42

npy_files= os.listdir('/content/target_npy_files/')
print(npy_files)

#load from disk
X = np.load('/content/target_npy_files/' + npy_files[0])
y = np.zeros(X.shape[0])
#print(npy_files[0])
#print(npy_files[1])
#print(npy_files[2])
print(X.shape)
print(y.shape)

In [0]:
for i, label in enumerate(npy_files[1:]):
   print(label)
   print(i)
   x = np.load('/content/target_npy_files/' + label)
   #print(x)
     

   X = np.vstack((X, x))
   print(X.shape)
   
   y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))
   print(y.shape)
   print(y)


In [8]:
split_ratio = 0.9
random_state = 42

npy_files= os.listdir('/content/target_npy_files/')
print(npy_files)

X = np.load('/content/target_npy_files/' + npy_files[0])
y = np.zeros(X.shape[0])
#print(npy_files[0])
#print(npy_files[1])
#print(npy_files[2])
print(X.shape)
print(y.shape)


# # Append all of the dataset into one single array
for i, label in enumerate(npy_files[1:]):
   x = np.load('/content/target_npy_files/' + label)
  
     

   X = np.vstack((X, x))
   y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))


# return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True) 



X_std = (X - np.mean(X)) / (np.std(X))
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)
np.save('/content/X_test.npy', X_test)
np.save('/content/y_test.npy', y_test)
X_train = X_train.reshape(X_train.shape[0], MFCC, 51, 1)
X_test = X_test.reshape(X_test.shape[0], MFCC, 51, 1)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

print(y_test)





['follow.npy', 'up.npy', 'left.npy', 'right.npy', 'forward.npy', 'no.npy', 'go.npy', 'yes.npy', 'stop.npy', 'down.npy']
(3158, 10, 51)
(3158,)
[3. 1. 1. ... 2. 4. 7.]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
np.save('/content/X_test', X_test)

In [0]:
X_train_flat = X_train.flatten()
X_train_max = max(X_train_flat)
X_train_min = min(X_train_flat)

print(X_train_max )
print(X_train_min)

In [0]:
def build_urban_model():
 return keras.Sequential([
# layer 1
# Convolutional layer      
    keras.layers.Conv2D(276, (4,MFCC), strides=(1,2), padding='same', activation='relu', input_shape=(MFCC,51,1), kernel_regularizer=keras.regularizers.l2(0.001)),     
# layer 2    
# Depthwise layer 
    keras.layers.SeparableConv2D(276, (3,3), strides=(2,2), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
# Separable layer 
    keras.layers.Conv2D(276, (1,1), strides=(1,1), padding='same', use_bias=False, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),

# layer 3
# Depthwise layer 
    keras.layers.SeparableConv2D(276, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
 # Separable
    keras.layers.Conv2D(276, (1,1), strides=(1,1), padding='same', use_bias=False, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),

# Average pool
    keras.layers.AveragePooling2D(pool_size=(5, 12), strides=(2,2), padding='valid', data_format=None),
# All in on vector
    keras.layers.Flatten(data_format=None),
# dropout to improve overfitting
    keras.layers.Dropout(0.5),
 # Place all filters together   
    keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001),  activation='relu'),
# Predict my word        
    keras.layers.Dense(10, activation='softmax')     
      
      
  ])  


In [0]:
def build_urban_model():
 return keras.Sequential([
      
    keras.layers.Conv2D(64, (4,MFCC), strides=(2,2), padding='same', activation='relu', input_shape=(MFCC,51,1), kernel_regularizer=keras.regularizers.l2(0.001)),     

# Depthwise layers
    keras.layers.SeparableConv2D(64, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Conv2D(64, (1,1), strides=(1,1), padding='same', use_bias=False, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),

# Depthwise layers
    keras.layers.SeparableConv2D(128, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Conv2D(128, (1,1), strides=(1,1), padding='same', use_bias=False, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),

# Depthwise layers
    keras.layers.SeparableConv2D(128, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Conv2D(128, (1,1), strides=(1,1), padding='same', use_bias=False, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),

    
    keras.layers.AveragePooling2D(pool_size=(5, 25), strides=(2,2), padding='valid', data_format=None),
    #keras.layers.Dropout(0.5),
    keras.layers.Flatten(data_format=None),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001),  activation='relu'),
  
      
    keras.layers.Dense(10, activation='softmax')     
      
      
  ])  

In [0]:
!ls -al target_npy_files
!rm -rf target_npy_files/.ipynb_checkpoints

In [0]:
import pandas as pd
import seaborn as sns; sns.set()


#classes=["follow", "down", "left", "stop", "no", "yes", "up", "forward", "go", "right"]
#classes=['forward', 'no', 'stop', 'follow', 'up', 'go', 'left', 'down', 'right', 'yes']
classes=['down', 'right', 'stop', 'yes', 'forward', 'follow', 'left', 'up', 'no', 'go']
!rm checkpoint
!rm checkpoints.data-00000-of-00001
!rm checkpoints.index
!rm checkpoints.meta


speech_graph = tf.Graph()
speech_sess = tf.Session(graph=speech_graph)

keras.backend.set_session(speech_sess)
with speech_graph.as_default():
     
  #build my model
  model = build_urban_model()
  #give me model structure
  model.summary()

  #my own optimizer
  Amartin = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
  
  model.compile(optimizer= "Adam", loss='categorical_crossentropy', metrics=['accuracy'])
  
    
  # run it
  history = model.fit(X_train, y_train_hot, batch_size=200, epochs=100, verbose=1, validation_data=(X_test, y_test_hot))



  # y_pred=model.predict_classes(X_test)
  # con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()   
  # real_label_batch = tf.argmax(y_test_hot, axis=1)
  # y_pred=model.predict_classes(X_test)
  # con_mat = tf.math.confusion_matrix(labels=real_label_batch, predictions=y_pred).numpy()   

  # con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
  # con_mat_df = pd.DataFrame(con_mat_norm,
  #                      index = classes, 
  #                      columns = classes)

  # figure = plt.figure(figsize=(6, 6))
  # sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
  # plt.tight_layout()
  # plt.ylabel('True label')
  # plt.xlabel('Predicted label')
  # plt.show()
  
  #save my graph  
  saver = tf.train.Saver()
  saver.save(speech_sess, 'checkpoints')

In [0]:
classes=["follow", "down", "left", "stop", "no", "yes", "up", "forward", "go", "right"]
real_label_batch = tf.argmax(y_test_hot, axis=1)
y_pred=model.predict_classes(X_test)
con_mat = tf.math.confusion_matrix(labels=real_label_batch, predictions=y_pred).numpy()   

con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)

figure = plt.figure(figsize=(6, 6))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()



tf.compat.v1.logging.info('Confusion Matrix:\n %s' % (con_mat))
tf.compat.v1.logging.info('Confusion Matrix Normalized:\n %s' % (con_mat_norm))

In [0]:
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(acc) + 1)

  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs,val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()

In [0]:
eval_speech_graph = tf.Graph()
eval_speech_sess = tf.Session(graph=eval_speech_graph)

keras.backend.set_session(eval_speech_sess)
with eval_speech_graph.as_default():
    keras.backend.set_learning_phase(0)
    eval_model = build_urban_model()

    #For quantization aware training only
    #tf.contrib.quantize.create_eval_graph(input_graph=eval_speech_graph)
    eval_speech_graph_def = eval_speech_graph.as_graph_def()
    saver = tf.train.Saver()
    saver.restore(eval_speech_sess, 'checkpoints')

    frozen_graph_def = tf.graph_util.convert_variables_to_constants( eval_speech_sess, eval_speech_graph_def, 
                                                                    [eval_model.output.op.name] )

    with open('speech_model.pb', 'wb') as f:
      f.write(frozen_graph_def.SerializeToString())

In [0]:
import pathlib

tf.enable_eager_execution()
tf.logging.set_verbosity(tf.logging.DEBUG)

converter = tf.lite.TFLiteConverter.from_frozen_graph( "speech_model.pb", ["conv2d_1_input"], ["dense_2/Softmax"])
tflite_model = converter.convert()

In [0]:
tflite_models_dir = pathlib.Path("/tmp/ESC50-split-models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [0]:
tflite_model_file = tflite_models_dir/"speech-model.tflite"
tflite_model_file.write_bytes(tflite_model)

In [0]:
!ls -lh {tflite_models_dir}

In [0]:
# Optimize for size
tf.logging.set_verbosity(tf.logging.INFO)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [0]:
# Create quantized values with an accurate dynamic range of activations, 
# for that need to provide a representative dataset

#sounds = tf.cast(X_train, tf.float32)/1000.0
sounds = tf.cast(X_train, tf.float32)
urban_ds = tf.data.Dataset.from_tensor_slices((sounds)).batch(1)
def representative_data_gen():
  for input_value in urban_ds.take(100):
    yield [input_value]
    
converter.representative_dataset = representative_data_gen

In [0]:
# tflite format quantized
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"speech_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

In [0]:
!ls -lh {tflite_models_dir}

In [0]:
# The converted model needs to be fully quantized. That means all ops need to be 
# quantized, no floats left. The input and outputs need to be integers too.

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()
tflite_model_quant_file = tflite_models_dir/"speech-model_quant_io.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

In [0]:
!ls -lh {tflite_models_dir}

In [10]:
# Load data for float model
sounds = tf.cast(X_test, tf.float32)
print(sounds.shape)
urban_ds = tf.data.Dataset.from_tensor_slices(sounds).batch(1)
#print(sounds)

X_train_flat = X_train.flatten()
X_train_max = max(X_train_flat)
X_train_min = min(X_train_flat)
print(X_train_max)
print(X_train_min)


# Load data for quantized model
X_test_flat = X_test.flatten()
X_test_max = max(X_test_flat)
X_test_min = min(X_test_flat)
print(X_test_max)
print(X_test_min)

quant_sounds = tf.quantization.quantize(X_test, X_train_min, X_train_max, tf.quint8, mode="MIN_COMBINED", round_mode="HALF_AWAY_FROM_ZERO", name=None)
#quant_sounds = tf.quantization.quantize(X_test, X_test_min, X_test_max, tf.quint8, mode="MIN_COMBINED", round_mode="HALF_AWAY_FROM_ZERO", name=None)
sounds_uint8 = tf.cast(quant_sounds[0], tf.uint8)
urban_ds_uint8 = tf.data.Dataset.from_tensor_slices(sounds_uint8).batch(1)

#print(sounds_uint8)
#print(X_test)
#print(quant_sounds[0])

(6819, 10, 51, 1)
2.58073663032399
-6.37757019552536
2.5334635916673305
-6.4010885108383695


In [15]:
if LOCAL_MODELS == False :
  if path.exists("/content/Speech-tiny/Models/speech/"):
    tflite_model="/content/Speech-tiny/Models/speech/speech-model.tflite"
    print('running external non-quantized speech model')
    interpreter = tf.lite.Interpreter(model_path=str(tflite_model))
    interpreter.allocate_tensors()
else:
  print('running local non-quantized model')
  interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
  interpreter.allocate_tensors()

running external non-quantized speech model


In [18]:
if LOCAL_MODELS == False :
  if path.exists("/content/Speech-tiny/Models/speech/"):
    tflite_model="/content/Speech-tiny/Models/speech/speech-model_quant_io.tflite"
    print('running external quantized speech model')
    interpreter = tf.lite.Interpreter(model_path=str(tflite_model))
    interpreter.allocate_tensors()
else:
  print('running local quantized model')
  interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
  interpreter_quant.allocate_tensors()

running external quantized speech model


In [0]:
def predict(records, n):
  max = records[0]
  
  for i in range(1, n):
    if records[i] > max:
      max = records[i]

  index = record.index(max)

  return max, index

In [0]:
#labels = ['forward', 'no', 'stop', 'follow', 'up', 'go', 'left', 'down', 'right', 'yes']

labels=['left', 'right', 'go', 'no', 'follow', 'up', 'stop', 'yes', 'down', 'forward']


for m in range(11, 30):
  print("\nRunning number", m)
  predict_number = m
  i=0
  for sound in urban_ds:
    #for list in sound:
      
      #break
    if i == predict_number:
      break
    i= i+1
    
    #print(list[1])
  #print(sound.shape)
  #print(list.shape)
  interpreter.set_tensor(interpreter.get_input_details()[0]["index"], sound)
  interpreter.invoke()
  predictions = interpreter.get_tensor(
      interpreter.get_output_details()[0]["index"])

  print(predictions)

  class_prediction = predictions.tolist()

  for record in class_prediction:
    record

  records = array(record)

  n = len(records)

  class_predicted, indice = predict(records, n)

  real_words = y_test[predict_number]
  real_word = int(real_words)
  word_predicted = labels[indice]

  print(labels)
  print("\nFLOATS")
  print('---------------------')
  print('Predicted class is ', labels[indice])
  print('Real class is ', labels[real_word])
  print('----------------------\n\n')


  i=0
  for sound in urban_ds_uint8:
    
    if i == predict_number:
      break
    i= i+1
  #print(sound)
  interpreter_quant.set_tensor(interpreter_quant.get_input_details()[0]["index"], sound)
  interpreter_quant.invoke()
  predictions = interpreter_quant.get_tensor(
      interpreter_quant.get_output_details()[0]["index"])


  class_prediction = predictions.tolist()

  for record in class_prediction:
    record

  records = array(record)

  n = len(records)

  class_predicted, indice = predict(records, n)

  print(predictions)

  print("\n\nINTEGERS")
  print("---------------------------")

  print('Predicted class is ', labels[indice])
  print('Real class is ', labels[real_word])
  print('----------------------------')